In [2]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI
# from langchain_community.vectorstores import Chroma
# from class langchain_chroma import Chroma

from langchain_chroma import Chroma

from langchain_openai import OpenAIEmbeddings
import os
from dotenv import load_dotenv

from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from operator import itemgetter
from langchain_core.output_parsers import StrOutputParser

from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain.schema.runnable import RunnablePassthrough

import json

from typing import Optional, List
from langchain.schema import Document


load_dotenv()
os.environ["OPENAI_API_KEY"] = os.getenv('OPENAI_KEY')

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")
# vectordb= Chroma(persist_directory="./UPDATE", embedding_function=embeddings, collection_name="UPDATE")
vectordb= Chroma(persist_directory="./Json1", embedding_function=embeddings, collection_name="Json1")




In [ ]:

class Search(BaseModel):
    """Search over a database"""

    query: str = Field(
        ..., description="Recherche par similarité appliquée à des documents d'informations sur l'école",
    )
    specialty: Optional[str] = Field(
        None, description="Spécialité à rechercher (exemple: MAIN, INFO, MATH)",
    )
    status: Optional[str] = Field(
        None, description="Statut du document, peut être 'publique' ou 'privé'",
    )

system = """Tu es un assistant machine, utilise le contexte fourni pour répondre poliment à la demande.\
si tu ne connais pas la réponse, dis que tu ne sais pas et redirige vers des sources publiques pertinentes."""

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "{question}"),
    ]
)

llm = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=0)
structured_llm = llm.with_structured_output(Search)
query_analyzer = {"question": RunnablePassthrough()} | prompt | structured_llm



def retrieval_single_field(search: Search, field: str, value) -> List[str]:
    """
    Recherche dans le vectordb en fonction d'un seul critère (title, specialty, status).
    Cette fonction est appelée pour chaque champ spécifique.
    """
    # Préparer le filtre en fonction du champ à partir des métadonnées
    _filter = {}
    if field == "Specialty":
        _filter["Specialty"] = search.specialty
    if field == "Status":
        _filter["Status"] = search.status
    
    results = vectordb.similarity_search(search.query, filter=_filter)
    # Retourner les identifiants des documents trouvés
    return [doc.metadata.get("ID", 'None') for doc in results]

def retrieval(search: Search) -> List[Document]:
    # Initialiser une liste pour stocker les identifiants de documents
    all_ids_set = set()

    # Recherche par specialty si spécifiée
    if search.specialty:
        all_ids_set.update(retrieval_single_field(search, "Specialty", search.specialty))

    # Recherche par status si spécifiée
    if search.status:
        all_ids_set.update(retrieval_single_field(search, "Status", search.status))


    else:
        # Si aucun filtre n'est fourni ou aucun résultat trouvé, retourner une liste vide
        return []
    

    all_ids = list(all_ids_set)
    print("all ids :", all_ids)
    print(all_ids)

    return all_ids



retrieval_chain = query_analyzer | retrieval

# def get_document_by_id(vector_store, doc_id):
#     results = vector_store.similarity_search(f"ID:{doc_id}")
#     print(results)
#     return results

def get_document_by_id(collection, vector_id):
    """
    Récupère un vecteur à partir de son ID dans une collection Chroma.

    Args:
        collection: L'objet Chroma collection où les vecteurs sont stockés.
        vector_id: L'ID unique associé au vecteur que vous souhaitez récupérer.

    Returns:
        Le vecteur et ses métadonnées associées, ou None si l'ID n'existe pas.
    """
    # Rechercher le vecteur correspondant à l'ID dans la collection
    results = collection.get(where={"ID": vector_id})
    
    if results and results["ids"]:
        # Retourner le vecteur et ses métadonnées
        # print(results)
        return results
    else:
        # ID non trouvé
        print(f"Aucun vecteur trouvé avec l'ID: {vector_id}")
        return None



In [10]:


query = "y a t il une maquette du programme de EISE ?"
# specialty = ["MAIN", "MTX"]
# specialty = "EISE (Électronique - Informatique Parcours systèmes embarqués)"

status = "privé"

# search_input = Search(query=query, status="publique", specialty=specialty)
search_input = Search(query=query, status="publique")



retrieved_documents = retrieval_chain.invoke(search_input)

print("#"*50)

for id in retrieved_documents :
    print("document :" , id)
    doc = get_document_by_id(vectordb, id)
    print(doc)
    print(f"Source : {doc['metadatas'][0].get('Source','')}")

    # print(f"Contenu : {doc['documents'][0]}")
    print(f"SPE : {doc['metadatas'][0].get('Speciality', 'no spec')}")


    # print(f"id : {doc['metadatas'][0].get('ID', 'pas d ID')}")
    print(f"Status : {doc['metadatas'][0].get('Status', 'pas de Satus')}")
    url = doc['metadatas'][0].get('URL', None)
    if(url) : print(f'URL : {url}')

    print("-" * 80)


all ids : ['b14ee77c-c766-4a22-aa54-632e5011c9d7', 'dc418144-4c76-4ef5-a0a2-72ecf3330817', '0f4b9511-d44d-42a8-afee-d1b95aa0bda6', '011a8f90-096e-4417-af60-d5dca48b19d8']
['b14ee77c-c766-4a22-aa54-632e5011c9d7', 'dc418144-4c76-4ef5-a0a2-72ecf3330817', '0f4b9511-d44d-42a8-afee-d1b95aa0bda6', '011a8f90-096e-4417-af60-d5dca48b19d8']
##################################################
document : b14ee77c-c766-4a22-aa54-632e5011c9d7
{'ids': ['2b2669c7-2c85-4e97-99a8-56acbe2fab79'], 'embeddings': None, 'documents': ["Le cycle ingénieur Electronique et Informatique (EI) est un cycle ingénieur (Bac+3 à Bac+5) de Polytech-Sorbonne. Cette spécialité forme des ingénieurs pour répondre aux besoins de l’industrie électronique et informatique dans les domaines des systèmes embarqués, objets connectés, sécurité, et innovation. Les systèmes embarqués, fruit de la miniaturisation des puces électroniques, révolutionnent l’industrie, l’environnement urbain, le transport et la médecine. La demande en ingén

# test alternatif

In [5]:

SPECIALTIES = ["AGRAL (Agroalimentaire)", "EISE (Électronique - Informatique Parcours systèmes embarqués)", "EI2I (Électronique - Informatique Parcours informatique industrielle)", "GM (Génie Mécanique)", "MAIN (Mathématiques appliquées et informatique)", "MTX (Matériaux - Chimie)", "ROB (Robotique)", "ST (Sciences de la terre : aménagement, environnement, énergie)"]

# results = vectordb.similarity_search(query="info",k=5,filter={"Speciality": "EISE (Électronique - Informatique Parcours systèmes embarqués)"})
results = vectordb.similarity_search(query="descriptif spécialité",k=5,filter={"Speciality": "AGRAL (Agroalimentaire)"})

# results = vectordb.similarity_search(query="Donne moi un exemple de ce qu'on étudie en MAIN",k=5,filter={"Speciality": "MAIN (Mathématiques appliquées et informatique)"})
# results = vectordb.similarity_search(query="Parle moi de la spécialité ROB",k=5,filter={"Status": "public"})


for doc in results :
    print(doc.metadata.get("ID", 'None'))
    print(doc.metadata.get("Speciality", 'None'))
    print(doc.metadata.get("Source", 'None'))
    print(doc.metadata.get("Status", 'None'))

    print("-"*80)



c5ddf018-c0eb-4f9d-8c34-0430b267cb64
AGRAL (Agroalimentaire)
/home/samuel/Bureau/ECOLE/Polyteque3/Projet_RAG/corpusv1/CORPUS_TOTAL(version samuel)/public/les_json_parfaits_en_vrac/descriptif-agral.json
publique
--------------------------------------------------------------------------------
dfa99e10-43f1-48ac-907b-7d8a246ef8b7
AGRAL (Agroalimentaire)
/home/samuel/Bureau/ECOLE/Polyteque3/Projet_RAG/corpusv1/CORPUS_TOTAL(version samuel)/public/les_json_parfaits_en_vrac/descriptif-agral.json
publique
--------------------------------------------------------------------------------


In [ ]:
def retrieval_single_field (query : str, _filter : str, vectordb : Chroma) :
    docs = vectordb.similarity_search(query,k=5,filter={"Speciality": _filter})
    # return [doc.metadata.get("ID", 'None') for doc in docs]
    # return [doc.metadata.get("Source", 'None') for doc in docs]
    return [(doc.metadata.get("ID", 'None'), doc.metadata.get("Source", 'None')) for doc in docs]


def retrieval(query : str, filter : list, vectordb : Chroma) -> List[Document]:
    # Initialiser une liste pour stocker les identifiants de documents
    all_ids_set = set()
    #on passe une liste de filtres (les spécialités)
    
    if filter :
        for _filter in filter :
            all_ids_set.update(retrieval_single_field(query, _filter, vectordb))
    else:
        # Si aucun filtre n'est fourni ou aucun résultat trouvé, retourner une liste vide
        return []
    

    all_ids = list(all_ids_set)
    # print("all ids :", all_ids)

    return all_ids


[('b4cb331d-5960-4167-8105-39c087b7d64f',
  '/home/samuel/Bureau/ECOLE/Polyteque3/Projet_RAG/corpusv1/CORPUS_TOTAL(version samuel)/pub/MAIN (Mathématiques appliquées et informatique)/MAIN3_maquette.pdf'),
 ('1df78ed3-3af5-4458-8b57-7b08b55aebc5',
  '/home/samuel/Bureau/ECOLE/Polyteque3/Projet_RAG/corpusv1/CORPUS_TOTAL(version samuel)/pub/MAIN (Mathématiques appliquées et informatique)/MAIN4_maquette.pdf'),
 ('53c54aa0-ed07-450e-bedc-c1c59c954d41',
  '/home/samuel/Bureau/ECOLE/Polyteque3/Projet_RAG/corpusv1/CORPUS_TOTAL(version samuel)/pub/EISE (Électronique - Informatique Parcours systèmes embarqués)/vie-pratique-5.json'),
 ('55651b00-875e-4cc7-91bc-ba98770f386f',
  '/home/samuel/Bureau/ECOLE/Polyteque3/Projet_RAG/corpusv1/CORPUS_TOTAL(version samuel)/public/EISE (Électronique - Informatique Parcours systèmes embarqués)/vie-pratique-5.json'),
 ('24a060bf-b2f6-418e-b430-69cb033a8ce7',
  '/home/samuel/Bureau/ECOLE/Polyteque3/Projet_RAG/corpusv1/CORPUS_TOTAL(version samuel)/private/MAIN (

In [ ]:
###COMPIL DE TOUT CE QUI EST EN DESSOUS SANS FAIRE DE CLASSE

#LES CLASSES
class MetadataFilters(BaseModel):
    """Query analyzer"""
    speciality: str | None = Field(None, description="Spécialités d'étude/diplômes mentionnées dans la question")
    document_title: str | None = Field(None, description="Titres de document mentionnés dans la question")



def filter_detection(question) -> dict :
    # Définition du prompt
    SPECIALTIES = ["AGRAL (Agroalimentaire)", "EISE (Électronique - Informatique Parcours systèmes embarqués)", "EI2I (Électronique - Informatique Parcours informatique industrielle)", "GM (Génie Mécanique)", "MAIN (Mathématiques appliquées et informatique)", "MTX (Matériaux - Chimie)", "ROB (Robotique)", "ST (Sciences de la terre : aménagement, environnement, énergie)"]
    llm = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=0)
    prompt = ChatPromptTemplate.from_messages([
        ("system", "Tu es un assistant chargé d'extraire des filtres de métadonnées à partir d'une question. Voici les filtres qui nous intéressent : specialty. plusieurs informations peuvent être ajoutés dans le même filtre"
        " Voici une liste des spécialités reconnues : {SPECIALTIES}."),

        ("human", "Voici la question de l'utilisateur : {question}. Retourne les informations au format JSON avec 'None' si rien ne correspond.")
    ])

    # Chaîne de traitement
    FilterChain = (prompt | llm | RunnablePassthrough())

    response = FilterChain.invoke({"question": question, "SPECIALTIES":SPECIALTIES})
    response = str(response.content)
    # Convertir le texte JSON en dictionnaire Python
    dict_answer = json.loads(response)
    # Afficher le dictionnaire
    print(dict_answer)
    return dict_answer

def flatten(xss) :
    return [x for xs in xss for x in xs]

def retrieval_single_field (query : str, _filter : str, vectordb : Chroma) :
    docs = vectordb.similarity_search(query,k=5,filter={"Speciality": _filter})
    # return [doc.metadata.get("ID", 'None') for doc in docs]
    # return [doc.metadata.get("Source", 'None') for doc in docs]
    return [doc.metadata.get("ID", 'None') for doc in docs]


def retrieval(query : str, filter : list, vectordb : Chroma) -> List[Document]:
    # Initialiser une liste pour stocker les identifiants de documents
    all_ids_set = set()
    #on passe une liste de filtres (les spécialités)
    
    if filter :
        for _filter in filter :
            all_ids_set.update(retrieval_single_field(query, _filter, vectordb))
    else:
        # Si aucun filtre n'est fourni ou aucun résultat trouvé, retourner une liste vide
        return []

    all_ids = list(all_ids_set)
    # print("all ids :", all_ids)

    return all_ids

def get_document_by_id(collection, vector_id):
    """
    Récupère un vecteur à partir de son ID dans une collection Chroma.

    Args:
        collection: L'objet Chroma collection où les vecteurs sont stockés.
        vector_id: L'ID unique associé au vecteur que vous souhaitez récupérer.

    Returns:
        Le vecteur et ses métadonnées associées, ou None si l'ID n'existe pas.
    """
    # Rechercher le vecteur correspondant à l'ID dans la collection
    results = collection.get(where={"ID": vector_id})
    
    if results and results["ids"]:
        # Retourner le vecteur et ses métadonnées
        # print(results)
        return results
    else:
        # ID non trouvé
        print(f"Aucun vecteur trouvé avec l'ID: {vector_id}")
        return None
    
def context_fetcher(retrieved_documents) :
    context = ""

    for id in retrieved_documents :
        doc = get_document_by_id(vectordb, id)
        context += f"Contenu : {doc['documents'][0]}\n"
        print(f"Contenu : {doc['documents'][0]}")

        context += f"Status : {doc['metadatas'][0].get('Status', 'pas de Satus')}\n"
        print(f"Status : {doc['metadatas'][0].get('Status', 'pas de Satus')}")

        context += f"SPE : {doc['metadatas'][0].get('Speciality', 'no spec')}\n"
        # print(f"SPE : {doc['metadatas'][0].get('Speciality', 'no spec')}")

        url = doc['metadatas'][0].get('URL', None)
        if(url) : 
            context += f'URL : {url}\n'
            # print(f'URL : {url}')
        context += "-" * 80 + '\n'

        print("-" * 80)
        
    return context
        

def total_process(question, vectordb) :

    ### D'abord, extraire les filtres de la question :
    filtered_ids = []
    dict_answer = filter_detection(question)

    spec = dict_answer.get('specialty','')
    # for specialty in spec :
    #     print(specialty)
    # print("#"*50)
    
    retrieved_documents = retrieval(question, spec, vectordb)
    chat_model = ChatOpenAI()
    output_parser = StrOutputParser()

    if retrieved_documents : #Si on a retrouvé des choses correspondantes
        # print("RESULTS WITH FILTERS DETECTED")
        context = context_fetcher(retrieved_documents)

        ### 3. Construction du prompt sous forme de template avec variables
        messages = [
            ("system", "Tu es un assistant machine. Utilise le contexte fourni pour répondre poliment à la demande.\nsi tu ne connait pas la réponse, dit que tu ne sais pas et redirige vers des sources publiques pertinentes."),
            ("user", "Query: {question}\n\nContext:\n{context}")
        ]

        # Ajout des filtres utilisés
        if spec:
            messages.append(("user", f"Voici les filtres utilisés : {', '.join(spec)}"))

        rag_prompt = ChatPromptTemplate.from_messages(messages)

        ### 4. Exécution du modèle
        
        RAG_chain = rag_prompt | chat_model | output_parser

        # Obtenir la réponse du modèle en fournissant les variables nécessaires
        response = RAG_chain.invoke({"question": question, "context": context})
    else:
        # print("NO RESULTS WITH FILTERS DETECTED RUNING CLASSIC")
        TEMPLATE = """\
    Tu es un assistant machine, utilise le contexte fourni pour répondre poliment à la demande.
    si tu ne connait pas la réponse, dit que tu ne sais pas et redirige vers des sources publiques pertinentes.

    Query:
    {question}

    Context:
    {context}
    """
        rag_prompt = ChatPromptTemplate.from_template(TEMPLATE)
        naive_retriever = vectordb.as_retriever(search_kwargs={ "k" : 10})
        setup_and_retrieval = RunnableParallel({"question": RunnablePassthrough(), "context": naive_retriever })
        output_parser = StrOutputParser()


        naive_retrieval_chain = setup_and_retrieval | rag_prompt | chat_model | output_parser
        response = naive_retrieval_chain.invoke(question)

    # ### 5. Affichage et retour du résultat
    # print("\n### Réponse générée ###\n", response)
    return response


In [8]:
# retrieval("Quelles spécialitées enseignent l'informatique ?", ["MAIN (Mathématiques appliquées et informatique)", "EISE (Électronique - Informatique Parcours systèmes embarqués)"], vectordb)

Answer = total_process("Où trouver des informations sur la spécialité Agral ?", vectordb)
print(Answer)

{'specialty': ['AGRAL (Agroalimentaire)']}
RESULTS WITH FILTERS DETECTED
Contenu : Le cycle ingénieur Agroalimentaire (AGRAL) est un cycle ingénieur (Bac+3 à Bac+5) de Polytech-Sorbonne. Cette formation apporte des compétences solides dans le domaine des sciences des aliments, les biotechnologies et le management industriel avec la démarche de projet comme outil pédagogique ce qui permet aux élèves de s’insérer dans un large champ de domaines et de fonctions. 
 L’industrie agroalimentaire constitue le premier secteur industriel français aussi bien en termes de chiffre d’affaires que d’emplois. L’une de ses forces est son ancrage sur l’ensemble du territoire français et même si le contexte est difficile, elle continue d’innover, d’exporter et surtout de créer des emplois. Pour relever ces défis l’objectif de la filière AGRAL à Polytech Sorbonne est, en conciliant la compétitivité nationale et internationale et le développement durable grâce à l’innovation, de certifier des ingénieurs ca